## getting the data from github
the first thing we want to do is get the data from github.  The following code pulls down the zipped psv file and loads it into the reta1960 object. 

In [23]:
import zipfile, io, requests
for i in range(1969,2013):
    request = requests.get('https://github.com/estheryou/fbi-reta-data/blob/master/recoded-data/reta_%s_data.csv.zip?raw=true'%(i))
    zfile = zipfile.ZipFile(io.BytesIO(request.content)) 
    test = zfile.open('reta_%s_data.csv'%(i)).read()
    a=open('reta_%s.csv'%(i), 'w') 
    a.write(test.decode("utf-8")) 
    a.close()


OSError: [Errno 28] No space left on device

## pushing the data into Google Big Query
The next step is to push the data into google big query. 
You can use this method: https://cloud.google.com/bigquery/loading-data-into-bigquery#loaddatapostrequest
Or you can figure out some other python-friendly, programmatic menthod.  
Also, don't forget that we have a custom delimiter ('|'), that will have to be set with the configuration.load.fieldDelimiter option.  This defines the custom separator for fields in a CSV file.  The default value is a comma (','). Since ('|') is not a comman, we need to set this option. BigQuery converts the string to ISO-8859-1 encoding, and then uses the first byte of the encoded string to split the data in its raw, binary state. 

In [5]:
import json
import csv
import os
import httplib2
import io 
from apiclient.discovery import build
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client import tools
from apiclient.http import MediaFileUpload 

#Step 1 Oauth2.0 Authorisation 
flow =flow_from_clientsecrets('client_secret.json',# the one downloaded from google console OAuth2.0 client ID
                           scope ='https://www.googleapis.com/auth/bigquery',
                           redirect_uri ='https://www.example.com/oauth2callback',
                            )
auth_uri = flow.step1_get_authorize_url() # Redirect the user to auth_uri
print ('Go to the following link in your browser: ' + auth_uri)
code = input('Enter verification code: ').strip()
credentials = flow.step2_exchange(code)

PROJECT_ID=input('Enter the Project_ID') # In the overview on the left of the developer console
DATASET_ID=input('Enter the Dataset_ID') # The dataset set in the bigquery 

http = httplib2.Http()
http1 = credentials.authorize(http)
print ('Authorisation successfully completed')
storage = Storage('bigquery_credentials.dat')
credentials = storage.get()

service=build('bigquery','v2',http=http1)# service contains the information about the project ,which will be used to 
                                         #request POST by jobs

# Step 2 Define the functions to be used later 

#fucntion start_and_wait
import json
import time
def start_and_wait(jobs, project_id, load, media_body=None):
  '''Run a load job with the given specification.
    jobs: client for the jobs collection in the service.
    project_id: project ID under which the job will run.
    load: the load job configuration.
    media_body: optional media object, ie file, to upload.
  '''
  start = time.time()
  job_id = 'ch06_%d' % start#might be something else 
  # Create the job.
  result = jobs.insert(
    projectId=project_id,
    body={
      'jobReference': {
        'jobId': job_id
      },
      'configuration': {
        'load': load
      }
},
    media_body=media_body).execute()
  print (json.dumps(result, indent=2))
  # Wait for completion.
  done = False
  while not done:
    time.sleep(10)
    result = jobs.get(projectId=project_id, jobId=job_id).execute()
    print ("%s %ds" % (result['status']['state'], time.time() - start))
    done = result['status']['state'] == 'DONE'
  # Print all errors and warnings.
  for err in result['status'].get('errors', []):
    print (json.dumps(err, indent=2))
  # Check for failure.
  if 'errorResult' in result['status']:
    print ('FAILED')
    print (json.dumps(result['status']['errorResult'], indent=2))
  else:
    print ('SUCCESS')
    

# Import class that handles file uploads.
from apiclient.http import MediaFileUpload

# Sample code authorization support.
# Functions to help run a load job.

def main():
 
  for i in range(1969,2013):
        import zipfile, io, requests
        request = requests.get('https://github.com/estheryou/fbi-reta-data/blob/master/recoded-data/reta_%s_data.csv.zip?raw=true'%(i))
        zfile = zipfile.ZipFile(io.BytesIO(request.content)) 
        test = zfile.open('reta_%s_data.csv'%(i)).read()
        a=open('reta_%s.csv'%i, 'w') 
        a.write(test.decode("utf-8")) 
        a.close()
        TABLE_ID='reta_%s'%(i)
         # Load configuration with the destination specified.
        load_config = {
          'destinationTable': {
          'projectId': PROJECT_ID,
          'datasetId': DATASET_ID,
      # You can update this for each example.
          'tableId': TABLE_ID
                               }
                       }
  # Setup the job here.
  # load[property] = value
        load_config['schema'] = {        
      'fields': [
         {'name':'id', 'type':'STRING'},
         {'name':'state', 'type':'STRING'},
         {'name':'ori_code', 'type':'STRING'},
         {'name':'city_group', 'type':'STRING'},
         {'name':'division', 'type':'STRING'},
         {'name':'year', 'type':'STRING'},
         {'name':'sequence', 'type':'STRING'},
         {'name':'juv_age', 'type':'STRING'},
         {'name':'core_ci', 'type':'STRING'},
         {'name':'covered_by', 'type':'STRING'},
         {'name':'ori_group', 'type':'STRING'},
         {'name':'last_update', 'type':'STRING'},
         {'name':'field_office', 'type':'STRING'},
         {'name':'num_months', 'type':'STRING'},
         {'name':'agency_count', 'type':'STRING'},
         {'name':'pop1', 'type':'INTEGER'},
         {'name':'county1', 'type':'STRING'},
         {'name':'msa1', 'type':'STRING'},
         {'name':'pop2', 'type':'INTEGER'},
         {'name':'county2', 'type':'STRING'},
         {'name':'msa2', 'type':'STRING'},
         {'name':'pop3', 'type':'INTEGER'},
         {'name':'county3', 'type':'STRING'},
         {'name':'msa3', 'type':'STRING'},
         {'name':'county1_pop', 'type':'INTEGER'},
         {'name':'county2_pop', 'type':'INTEGER'},
         {'name':'county3_pop', 'type':'INTEGER'},
         {'name':'pop_source', 'type':'STRING'},
         {'name':'follow_up', 'type':'STRING'},
         {'name':'special_mail_group', 'type':'STRING'},
         {'name':'special_mail_addr', 'type':'STRING'},
         {'name':'agency_name', 'type':'STRING'},
         {'name':'agency_state', 'type':'STRING'},
         {'name':'address1', 'type':'STRING'},
         {'name':'address2', 'type':'STRING'},
         {'name':'address3', 'type':'STRING'},
         {'name':'address4', 'type':'STRING'},
         {'name':'zip', 'type':'STRING'},
         {'name':'old_pop_group', 'type':'STRING'},
         {'name':'unused_header', 'type':'STRING'},
         {'name':'jan_month_in', 'type':'STRING'},
         {'name':'jan_date_last_update', 'type':'STRING'},
         {'name':'jan_card_0_type', 'type':'STRING'},
         {'name':'jan_card_1_type', 'type':'STRING'},
         {'name':'jan_card_2_type', 'type':'STRING'},
         {'name':'jan_card_3_type', 'type':'STRING'},
         {'name':'jan_card_4_type', 'type':'STRING'},
         {'name':'jan_card_0_pt', 'type':'STRING'},
         {'name':'jan_card_1_pt', 'type':'STRING'},
         {'name':'jan_card_2_pt', 'type':'STRING'},
         {'name':'jan_card_3_pt', 'type':'STRING'},
         {'name':'jan_unfounded_murder', 'type':'INTEGER'},
         {'name':'jan_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'jan_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'jan_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'jan_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'jan_unfounded_robbery', 'type':'INTEGER'},
         {'name':'jan_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'jan_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'jan_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'jan_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jan_unfounded_assault', 'type':'INTEGER'},
         {'name':'jan_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'jan_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'jan_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'jan_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'jan_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'jan_unfounded_burglary', 'type':'INTEGER'},
         {'name':'jan_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'jan_unfounded_larceny', 'type':'INTEGER'},
         {'name':'jan_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'jan_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jan_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'jan_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jan_unfounded_unused', 'type':'INTEGER'},
         {'name':'jan_actual_murder', 'type':'INTEGER'},
         {'name':'jan_actual_manslaughter', 'type':'INTEGER'},
         {'name':'jan_actual_rape_total', 'type':'INTEGER'},
         {'name':'jan_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'jan_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'jan_actual_robbery', 'type':'INTEGER'},
         {'name':'jan_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'jan_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'jan_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'jan_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jan_actual_assault', 'type':'INTEGER'},
         {'name':'jan_actual_assault_gun', 'type':'INTEGER'},
         {'name':'jan_actual_assault_knife', 'type':'INTEGER'},
         {'name':'jan_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'jan_actual_assault_hands', 'type':'INTEGER'},
         {'name':'jan_actual_assault_simple', 'type':'INTEGER'},
         {'name':'jan_actual_burglary', 'type':'INTEGER'},
         {'name':'jan_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'jan_actual_larceny', 'type':'INTEGER'},
         {'name':'jan_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_actual_auto_theft', 'type':'INTEGER'},
         {'name':'jan_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jan_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'jan_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jan_actual_unused', 'type':'INTEGER'},
         {'name':'jan_cleared_murder', 'type':'INTEGER'},
         {'name':'jan_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'jan_cleared_rape_total', 'type':'INTEGER'},
         {'name':'jan_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'jan_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'jan_cleared_robbery', 'type':'INTEGER'},
         {'name':'jan_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'jan_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'jan_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'jan_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jan_cleared_assault', 'type':'INTEGER'},
         {'name':'jan_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'jan_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'jan_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'jan_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'jan_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'jan_cleared_burglary', 'type':'INTEGER'},
         {'name':'jan_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'jan_cleared_larceny', 'type':'INTEGER'},
         {'name':'jan_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'jan_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jan_cleared_unused', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jan_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'jan_officers_killed_felony', 'type':'STRING'},
         {'name':'jan_officers_killed_accident', 'type':'STRING'},
         {'name':'jan_officers_assaulted', 'type':'STRING'},
         {'name':'feb_month_in', 'type':'STRING'},
         {'name':'feb_date_last_update', 'type':'STRING'},
         {'name':'feb_card_0_type', 'type':'STRING'},
         {'name':'feb_card_1_type', 'type':'STRING'},
         {'name':'feb_card_2_type', 'type':'STRING'},
         {'name':'feb_card_3_type', 'type':'STRING'},
         {'name':'feb_card_4_type', 'type':'STRING'},
         {'name':'feb_card_0_pt', 'type':'STRING'},
         {'name':'feb_card_1_pt', 'type':'STRING'},
         {'name':'feb_card_2_pt', 'type':'STRING'},
         {'name':'feb_card_3_pt', 'type':'STRING'},
         {'name':'feb_unfounded_murder', 'type':'INTEGER'},
         {'name':'feb_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'feb_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'feb_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'feb_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'feb_unfounded_robbery', 'type':'INTEGER'},
         {'name':'feb_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'feb_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'feb_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'feb_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'feb_unfounded_assault', 'type':'INTEGER'},
         {'name':'feb_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'feb_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'feb_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'feb_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'feb_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'feb_unfounded_burglary', 'type':'INTEGER'},
         {'name':'feb_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'feb_unfounded_larceny', 'type':'INTEGER'},
         {'name':'feb_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'feb_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'feb_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'feb_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'feb_unfounded_unused', 'type':'INTEGER'},
         {'name':'feb_actual_murder', 'type':'INTEGER'},
         {'name':'feb_actual_manslaughter', 'type':'INTEGER'},
         {'name':'feb_actual_rape_total', 'type':'INTEGER'},
         {'name':'feb_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'feb_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'feb_actual_robbery', 'type':'INTEGER'},
         {'name':'feb_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'feb_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'feb_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'feb_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'feb_actual_assault', 'type':'INTEGER'},
         {'name':'feb_actual_assault_gun', 'type':'INTEGER'},
         {'name':'feb_actual_assault_knife', 'type':'INTEGER'},
         {'name':'feb_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'feb_actual_assault_hands', 'type':'INTEGER'},
         {'name':'feb_actual_assault_simple', 'type':'INTEGER'},
         {'name':'feb_actual_burglary', 'type':'INTEGER'},
         {'name':'feb_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'feb_actual_larceny', 'type':'INTEGER'},
         {'name':'feb_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_actual_auto_theft', 'type':'INTEGER'},
         {'name':'feb_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'feb_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'feb_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'feb_actual_unused', 'type':'INTEGER'},
         {'name':'feb_cleared_murder', 'type':'INTEGER'},
         {'name':'feb_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'feb_cleared_rape_total', 'type':'INTEGER'},
         {'name':'feb_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'feb_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'feb_cleared_robbery', 'type':'INTEGER'},
         {'name':'feb_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'feb_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'feb_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'feb_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'feb_cleared_assault', 'type':'INTEGER'},
         {'name':'feb_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'feb_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'feb_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'feb_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'feb_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'feb_cleared_burglary', 'type':'INTEGER'},
         {'name':'feb_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'feb_cleared_larceny', 'type':'INTEGER'},
         {'name':'feb_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'feb_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'feb_cleared_unused', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'feb_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'feb_officers_killed_felony', 'type':'STRING'},
         {'name':'feb_officers_killed_accident', 'type':'STRING'},
         {'name':'feb_officers_assaulted', 'type':'STRING'},
         {'name':'mar_month_in', 'type':'STRING'},
         {'name':'mar_date_last_update', 'type':'STRING'},
         {'name':'mar_card_0_type', 'type':'STRING'},
         {'name':'mar_card_1_type', 'type':'STRING'},
         {'name':'mar_card_2_type', 'type':'STRING'},
         {'name':'mar_card_3_type', 'type':'STRING'},
         {'name':'mar_card_4_type', 'type':'STRING'},
         {'name':'mar_card_0_pt', 'type':'STRING'},
         {'name':'mar_card_1_pt', 'type':'STRING'},
         {'name':'mar_card_2_pt', 'type':'STRING'},
         {'name':'mar_card_3_pt', 'type':'STRING'},
         {'name':'mar_unfounded_murder', 'type':'INTEGER'},
         {'name':'mar_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'mar_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'mar_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'mar_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'mar_unfounded_robbery', 'type':'INTEGER'},
         {'name':'mar_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'mar_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'mar_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'mar_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'mar_unfounded_assault', 'type':'INTEGER'},
         {'name':'mar_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'mar_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'mar_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'mar_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'mar_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'mar_unfounded_burglary', 'type':'INTEGER'},
         {'name':'mar_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'mar_unfounded_larceny', 'type':'INTEGER'},
         {'name':'mar_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'mar_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'mar_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'mar_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'mar_unfounded_unused', 'type':'INTEGER'},
         {'name':'mar_actual_murder', 'type':'INTEGER'},
         {'name':'mar_actual_manslaughter', 'type':'INTEGER'},
         {'name':'mar_actual_rape_total', 'type':'INTEGER'},
         {'name':'mar_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'mar_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'mar_actual_robbery', 'type':'INTEGER'},
         {'name':'mar_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'mar_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'mar_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'mar_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'mar_actual_assault', 'type':'INTEGER'},
         {'name':'mar_actual_assault_gun', 'type':'INTEGER'},
         {'name':'mar_actual_assault_knife', 'type':'INTEGER'},
         {'name':'mar_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'mar_actual_assault_hands', 'type':'INTEGER'},
         {'name':'mar_actual_assault_simple', 'type':'INTEGER'},
         {'name':'mar_actual_burglary', 'type':'INTEGER'},
         {'name':'mar_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'mar_actual_larceny', 'type':'INTEGER'},
         {'name':'mar_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_actual_auto_theft', 'type':'INTEGER'},
         {'name':'mar_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'mar_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'mar_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'mar_actual_unused', 'type':'INTEGER'},
         {'name':'mar_cleared_murder', 'type':'INTEGER'},
         {'name':'mar_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'mar_cleared_rape_total', 'type':'INTEGER'},
         {'name':'mar_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'mar_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'mar_cleared_robbery', 'type':'INTEGER'},
         {'name':'mar_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'mar_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'mar_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'mar_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'mar_cleared_assault', 'type':'INTEGER'},
         {'name':'mar_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'mar_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'mar_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'mar_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'mar_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'mar_cleared_burglary', 'type':'INTEGER'},
         {'name':'mar_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'mar_cleared_larceny', 'type':'INTEGER'},
         {'name':'mar_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'mar_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'mar_cleared_unused', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'mar_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'mar_officers_killed_felony', 'type':'STRING'},
         {'name':'mar_officers_killed_accident', 'type':'STRING'},
         {'name':'mar_officers_assaulted', 'type':'STRING'},
         {'name':'apr_month_in', 'type':'STRING'},
         {'name':'apr_date_last_update', 'type':'STRING'},
         {'name':'apr_card_0_type', 'type':'STRING'},
         {'name':'apr_card_1_type', 'type':'STRING'},
         {'name':'apr_card_2_type', 'type':'STRING'},
         {'name':'apr_card_3_type', 'type':'STRING'},
         {'name':'apr_card_4_type', 'type':'STRING'},
         {'name':'apr_card_0_pt', 'type':'STRING'},
         {'name':'apr_card_1_pt', 'type':'STRING'},
         {'name':'apr_card_2_pt', 'type':'STRING'},
         {'name':'apr_card_3_pt', 'type':'STRING'},
         {'name':'apr_unfounded_murder', 'type':'INTEGER'},
         {'name':'apr_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'apr_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'apr_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'apr_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'apr_unfounded_robbery', 'type':'INTEGER'},
         {'name':'apr_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'apr_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'apr_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'apr_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'apr_unfounded_assault', 'type':'INTEGER'},
         {'name':'apr_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'apr_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'apr_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'apr_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'apr_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'apr_unfounded_burglary', 'type':'INTEGER'},
         {'name':'apr_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'apr_unfounded_larceny', 'type':'INTEGER'},
         {'name':'apr_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'apr_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'apr_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'apr_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'apr_unfounded_unused', 'type':'INTEGER'},
         {'name':'apr_actual_murder', 'type':'INTEGER'},
         {'name':'apr_actual_manslaughter', 'type':'INTEGER'},
         {'name':'apr_actual_rape_total', 'type':'INTEGER'},
         {'name':'apr_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'apr_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'apr_actual_robbery', 'type':'INTEGER'},
         {'name':'apr_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'apr_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'apr_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'apr_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'apr_actual_assault', 'type':'INTEGER'},
         {'name':'apr_actual_assault_gun', 'type':'INTEGER'},
         {'name':'apr_actual_assault_knife', 'type':'INTEGER'},
         {'name':'apr_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'apr_actual_assault_hands', 'type':'INTEGER'},
         {'name':'apr_actual_assault_simple', 'type':'INTEGER'},
         {'name':'apr_actual_burglary', 'type':'INTEGER'},
         {'name':'apr_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'apr_actual_larceny', 'type':'INTEGER'},
         {'name':'apr_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_actual_auto_theft', 'type':'INTEGER'},
         {'name':'apr_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'apr_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'apr_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'apr_actual_unused', 'type':'INTEGER'},
         {'name':'apr_cleared_murder', 'type':'INTEGER'},
         {'name':'apr_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'apr_cleared_rape_total', 'type':'INTEGER'},
         {'name':'apr_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'apr_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'apr_cleared_robbery', 'type':'INTEGER'},
         {'name':'apr_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'apr_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'apr_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'apr_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'apr_cleared_assault', 'type':'INTEGER'},
         {'name':'apr_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'apr_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'apr_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'apr_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'apr_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'apr_cleared_burglary', 'type':'INTEGER'},
         {'name':'apr_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'apr_cleared_larceny', 'type':'INTEGER'},
         {'name':'apr_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'apr_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'apr_cleared_unused', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'apr_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'apr_officers_killed_felony', 'type':'STRING'},
         {'name':'apr_officers_killed_accident', 'type':'STRING'},
         {'name':'apr_officers_assaulted', 'type':'STRING'},
         {'name':'may_month_in', 'type':'STRING'},
         {'name':'may_date_last_update', 'type':'STRING'},
         {'name':'may_card_0_type', 'type':'STRING'},
         {'name':'may_card_1_type', 'type':'STRING'},
         {'name':'may_card_2_type', 'type':'STRING'},
         {'name':'may_card_3_type', 'type':'STRING'},
         {'name':'may_card_4_type', 'type':'STRING'},
         {'name':'may_card_0_pt', 'type':'STRING'},
         {'name':'may_card_1_pt', 'type':'STRING'},
         {'name':'may_card_2_pt', 'type':'STRING'},
         {'name':'may_card_3_pt', 'type':'STRING'},
         {'name':'may_unfounded_murder', 'type':'INTEGER'},
         {'name':'may_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'may_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'may_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'may_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'may_unfounded_robbery', 'type':'INTEGER'},
         {'name':'may_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'may_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'may_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'may_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'may_unfounded_assault', 'type':'INTEGER'},
         {'name':'may_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'may_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'may_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'may_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'may_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'may_unfounded_burglary', 'type':'INTEGER'},
         {'name':'may_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'may_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'may_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'may_unfounded_larceny', 'type':'INTEGER'},
         {'name':'may_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'may_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'may_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'may_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'may_unfounded_unused', 'type':'INTEGER'},
         {'name':'may_actual_murder', 'type':'INTEGER'},
         {'name':'may_actual_manslaughter', 'type':'INTEGER'},
         {'name':'may_actual_rape_total', 'type':'INTEGER'},
         {'name':'may_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'may_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'may_actual_robbery', 'type':'INTEGER'},
         {'name':'may_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'may_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'may_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'may_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'may_actual_assault', 'type':'INTEGER'},
         {'name':'may_actual_assault_gun', 'type':'INTEGER'},
         {'name':'may_actual_assault_knife', 'type':'INTEGER'},
         {'name':'may_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'may_actual_assault_hands', 'type':'INTEGER'},
         {'name':'may_actual_assault_simple', 'type':'INTEGER'},
         {'name':'may_actual_burglary', 'type':'INTEGER'},
         {'name':'may_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'may_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'may_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'may_actual_larceny', 'type':'INTEGER'},
         {'name':'may_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_actual_auto_theft', 'type':'INTEGER'},
         {'name':'may_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'may_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'may_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'may_actual_unused', 'type':'INTEGER'},
         {'name':'may_cleared_murder', 'type':'INTEGER'},
         {'name':'may_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'may_cleared_rape_total', 'type':'INTEGER'},
         {'name':'may_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'may_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'may_cleared_robbery', 'type':'INTEGER'},
         {'name':'may_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'may_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'may_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'may_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'may_cleared_assault', 'type':'INTEGER'},
         {'name':'may_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'may_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'may_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'may_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'may_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'may_cleared_burglary', 'type':'INTEGER'},
         {'name':'may_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'may_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'may_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'may_cleared_larceny', 'type':'INTEGER'},
         {'name':'may_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'may_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'may_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'may_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'may_cleared_unused', 'type':'INTEGER'},
         {'name':'may_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'may_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'may_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'may_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'may_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'may_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'may_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'may_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'may_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'may_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'may_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'may_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'may_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'may_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'may_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'may_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'may_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'may_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'may_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'may_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'may_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'may_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'may_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'may_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'may_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'may_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'may_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'may_officers_killed_felony', 'type':'STRING'},
         {'name':'may_officers_killed_accident', 'type':'STRING'},
         {'name':'may_officers_assaulted', 'type':'STRING'},
         {'name':'jun_month_in', 'type':'STRING'},
         {'name':'jun_date_last_update', 'type':'STRING'},
         {'name':'jun_card_0_type', 'type':'STRING'},
         {'name':'jun_card_1_type', 'type':'STRING'},
         {'name':'jun_card_2_type', 'type':'STRING'},
         {'name':'jun_card_3_type', 'type':'STRING'},
         {'name':'jun_card_4_type', 'type':'STRING'},
         {'name':'jun_card_0_pt', 'type':'STRING'},
         {'name':'jun_card_1_pt', 'type':'STRING'},
         {'name':'jun_card_2_pt', 'type':'STRING'},
         {'name':'jun_card_3_pt', 'type':'STRING'},
         {'name':'jun_unfounded_murder', 'type':'INTEGER'},
         {'name':'jun_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'jun_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'jun_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'jun_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'jun_unfounded_robbery', 'type':'INTEGER'},
         {'name':'jun_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'jun_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'jun_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'jun_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jun_unfounded_assault', 'type':'INTEGER'},
         {'name':'jun_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'jun_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'jun_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'jun_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'jun_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'jun_unfounded_burglary', 'type':'INTEGER'},
         {'name':'jun_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'jun_unfounded_larceny', 'type':'INTEGER'},
         {'name':'jun_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'jun_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jun_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'jun_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jun_unfounded_unused', 'type':'INTEGER'},
         {'name':'jun_actual_murder', 'type':'INTEGER'},
         {'name':'jun_actual_manslaughter', 'type':'INTEGER'},
         {'name':'jun_actual_rape_total', 'type':'INTEGER'},
         {'name':'jun_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'jun_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'jun_actual_robbery', 'type':'INTEGER'},
         {'name':'jun_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'jun_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'jun_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'jun_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jun_actual_assault', 'type':'INTEGER'},
         {'name':'jun_actual_assault_gun', 'type':'INTEGER'},
         {'name':'jun_actual_assault_knife', 'type':'INTEGER'},
         {'name':'jun_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'jun_actual_assault_hands', 'type':'INTEGER'},
         {'name':'jun_actual_assault_simple', 'type':'INTEGER'},
         {'name':'jun_actual_burglary', 'type':'INTEGER'},
         {'name':'jun_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'jun_actual_larceny', 'type':'INTEGER'},
         {'name':'jun_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_actual_auto_theft', 'type':'INTEGER'},
         {'name':'jun_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jun_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'jun_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jun_actual_unused', 'type':'INTEGER'},
         {'name':'jun_cleared_murder', 'type':'INTEGER'},
         {'name':'jun_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'jun_cleared_rape_total', 'type':'INTEGER'},
         {'name':'jun_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'jun_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'jun_cleared_robbery', 'type':'INTEGER'},
         {'name':'jun_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'jun_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'jun_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'jun_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jun_cleared_assault', 'type':'INTEGER'},
         {'name':'jun_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'jun_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'jun_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'jun_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'jun_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'jun_cleared_burglary', 'type':'INTEGER'},
         {'name':'jun_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'jun_cleared_larceny', 'type':'INTEGER'},
         {'name':'jun_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'jun_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jun_cleared_unused', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jun_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'jun_officers_killed_felony', 'type':'STRING'},
         {'name':'jun_officers_killed_accident', 'type':'STRING'},
         {'name':'jun_officers_assaulted', 'type':'STRING'},
         {'name':'jul_month_in', 'type':'STRING'},
         {'name':'jul_date_last_update', 'type':'STRING'},
         {'name':'jul_card_0_type', 'type':'STRING'},
         {'name':'jul_card_1_type', 'type':'STRING'},
         {'name':'jul_card_2_type', 'type':'STRING'},
         {'name':'jul_card_3_type', 'type':'STRING'},
         {'name':'jul_card_4_type', 'type':'STRING'},
         {'name':'jul_card_0_pt', 'type':'STRING'},
         {'name':'jul_card_1_pt', 'type':'STRING'},
         {'name':'jul_card_2_pt', 'type':'STRING'},
         {'name':'jul_card_3_pt', 'type':'STRING'},
         {'name':'jul_unfounded_murder', 'type':'INTEGER'},
         {'name':'jul_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'jul_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'jul_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'jul_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'jul_unfounded_robbery', 'type':'INTEGER'},
         {'name':'jul_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'jul_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'jul_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'jul_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jul_unfounded_assault', 'type':'INTEGER'},
         {'name':'jul_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'jul_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'jul_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'jul_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'jul_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'jul_unfounded_burglary', 'type':'INTEGER'},
         {'name':'jul_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'jul_unfounded_larceny', 'type':'INTEGER'},
         {'name':'jul_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'jul_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jul_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'jul_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jul_unfounded_unused', 'type':'INTEGER'},
         {'name':'jul_actual_murder', 'type':'INTEGER'},
         {'name':'jul_actual_manslaughter', 'type':'INTEGER'},
         {'name':'jul_actual_rape_total', 'type':'INTEGER'},
         {'name':'jul_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'jul_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'jul_actual_robbery', 'type':'INTEGER'},
         {'name':'jul_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'jul_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'jul_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'jul_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jul_actual_assault', 'type':'INTEGER'},
         {'name':'jul_actual_assault_gun', 'type':'INTEGER'},
         {'name':'jul_actual_assault_knife', 'type':'INTEGER'},
         {'name':'jul_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'jul_actual_assault_hands', 'type':'INTEGER'},
         {'name':'jul_actual_assault_simple', 'type':'INTEGER'},
         {'name':'jul_actual_burglary', 'type':'INTEGER'},
         {'name':'jul_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'jul_actual_larceny', 'type':'INTEGER'},
         {'name':'jul_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_actual_auto_theft', 'type':'INTEGER'},
         {'name':'jul_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jul_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'jul_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jul_actual_unused', 'type':'INTEGER'},
         {'name':'jul_cleared_murder', 'type':'INTEGER'},
         {'name':'jul_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'jul_cleared_rape_total', 'type':'INTEGER'},
         {'name':'jul_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'jul_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'jul_cleared_robbery', 'type':'INTEGER'},
         {'name':'jul_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'jul_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'jul_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'jul_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jul_cleared_assault', 'type':'INTEGER'},
         {'name':'jul_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'jul_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'jul_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'jul_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'jul_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'jul_cleared_burglary', 'type':'INTEGER'},
         {'name':'jul_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'jul_cleared_larceny', 'type':'INTEGER'},
         {'name':'jul_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'jul_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jul_cleared_unused', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'jul_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'jul_officers_killed_felony', 'type':'STRING'},
         {'name':'jul_officers_killed_accident', 'type':'STRING'},
         {'name':'jul_officers_assaulted', 'type':'STRING'},
         {'name':'aug_month_in', 'type':'STRING'},
         {'name':'aug_date_last_update', 'type':'STRING'},
         {'name':'aug_card_0_type', 'type':'STRING'},
         {'name':'aug_card_1_type', 'type':'STRING'},
         {'name':'aug_card_2_type', 'type':'STRING'},
         {'name':'aug_card_3_type', 'type':'STRING'},
         {'name':'aug_card_4_type', 'type':'STRING'},
         {'name':'aug_card_0_pt', 'type':'STRING'},
         {'name':'aug_card_1_pt', 'type':'STRING'},
         {'name':'aug_card_2_pt', 'type':'STRING'},
         {'name':'aug_card_3_pt', 'type':'STRING'},
         {'name':'aug_unfounded_murder', 'type':'INTEGER'},
         {'name':'aug_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'aug_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'aug_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'aug_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'aug_unfounded_robbery', 'type':'INTEGER'},
         {'name':'aug_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'aug_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'aug_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'aug_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'aug_unfounded_assault', 'type':'INTEGER'},
         {'name':'aug_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'aug_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'aug_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'aug_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'aug_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'aug_unfounded_burglary', 'type':'INTEGER'},
         {'name':'aug_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'aug_unfounded_larceny', 'type':'INTEGER'},
         {'name':'aug_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'aug_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'aug_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'aug_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'aug_unfounded_unused', 'type':'INTEGER'},
         {'name':'aug_actual_murder', 'type':'INTEGER'},
         {'name':'aug_actual_manslaughter', 'type':'INTEGER'},
         {'name':'aug_actual_rape_total', 'type':'INTEGER'},
         {'name':'aug_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'aug_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'aug_actual_robbery', 'type':'INTEGER'},
         {'name':'aug_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'aug_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'aug_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'aug_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'aug_actual_assault', 'type':'INTEGER'},
         {'name':'aug_actual_assault_gun', 'type':'INTEGER'},
         {'name':'aug_actual_assault_knife', 'type':'INTEGER'},
         {'name':'aug_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'aug_actual_assault_hands', 'type':'INTEGER'},
         {'name':'aug_actual_assault_simple', 'type':'INTEGER'},
         {'name':'aug_actual_burglary', 'type':'INTEGER'},
         {'name':'aug_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'aug_actual_larceny', 'type':'INTEGER'},
         {'name':'aug_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_actual_auto_theft', 'type':'INTEGER'},
         {'name':'aug_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'aug_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'aug_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'aug_actual_unused', 'type':'INTEGER'},
         {'name':'aug_cleared_murder', 'type':'INTEGER'},
         {'name':'aug_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'aug_cleared_rape_total', 'type':'INTEGER'},
         {'name':'aug_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'aug_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'aug_cleared_robbery', 'type':'INTEGER'},
         {'name':'aug_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'aug_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'aug_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'aug_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'aug_cleared_assault', 'type':'INTEGER'},
         {'name':'aug_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'aug_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'aug_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'aug_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'aug_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'aug_cleared_burglary', 'type':'INTEGER'},
         {'name':'aug_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'aug_cleared_larceny', 'type':'INTEGER'},
         {'name':'aug_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'aug_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'aug_cleared_unused', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'aug_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'aug_officers_killed_felony', 'type':'STRING'},
         {'name':'aug_officers_killed_accident', 'type':'STRING'},
         {'name':'aug_officers_assaulted', 'type':'STRING'},
         {'name':'sep_month_in', 'type':'STRING'},
         {'name':'sep_date_last_update', 'type':'STRING'},
         {'name':'sep_card_0_type', 'type':'STRING'},
         {'name':'sep_card_1_type', 'type':'STRING'},
         {'name':'sep_card_2_type', 'type':'STRING'},
         {'name':'sep_card_3_type', 'type':'STRING'},
         {'name':'sep_card_4_type', 'type':'STRING'},
         {'name':'sep_card_0_pt', 'type':'STRING'},
         {'name':'sep_card_1_pt', 'type':'STRING'},
         {'name':'sep_card_2_pt', 'type':'STRING'},
         {'name':'sep_card_3_pt', 'type':'STRING'},
         {'name':'sep_unfounded_murder', 'type':'INTEGER'},
         {'name':'sep_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'sep_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'sep_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'sep_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'sep_unfounded_robbery', 'type':'INTEGER'},
         {'name':'sep_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'sep_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'sep_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'sep_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'sep_unfounded_assault', 'type':'INTEGER'},
         {'name':'sep_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'sep_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'sep_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'sep_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'sep_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'sep_unfounded_burglary', 'type':'INTEGER'},
         {'name':'sep_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'sep_unfounded_larceny', 'type':'INTEGER'},
         {'name':'sep_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'sep_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'sep_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'sep_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'sep_unfounded_unused', 'type':'INTEGER'},
         {'name':'sep_actual_murder', 'type':'INTEGER'},
         {'name':'sep_actual_manslaughter', 'type':'INTEGER'},
         {'name':'sep_actual_rape_total', 'type':'INTEGER'},
         {'name':'sep_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'sep_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'sep_actual_robbery', 'type':'INTEGER'},
         {'name':'sep_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'sep_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'sep_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'sep_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'sep_actual_assault', 'type':'INTEGER'},
         {'name':'sep_actual_assault_gun', 'type':'INTEGER'},
         {'name':'sep_actual_assault_knife', 'type':'INTEGER'},
         {'name':'sep_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'sep_actual_assault_hands', 'type':'INTEGER'},
         {'name':'sep_actual_assault_simple', 'type':'INTEGER'},
         {'name':'sep_actual_burglary', 'type':'INTEGER'},
         {'name':'sep_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'sep_actual_larceny', 'type':'INTEGER'},
         {'name':'sep_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_actual_auto_theft', 'type':'INTEGER'},
         {'name':'sep_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'sep_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'sep_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'sep_actual_unused', 'type':'INTEGER'},
         {'name':'sep_cleared_murder', 'type':'INTEGER'},
         {'name':'sep_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'sep_cleared_rape_total', 'type':'INTEGER'},
         {'name':'sep_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'sep_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'sep_cleared_robbery', 'type':'INTEGER'},
         {'name':'sep_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'sep_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'sep_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'sep_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'sep_cleared_assault', 'type':'INTEGER'},
         {'name':'sep_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'sep_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'sep_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'sep_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'sep_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'sep_cleared_burglary', 'type':'INTEGER'},
         {'name':'sep_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'sep_cleared_larceny', 'type':'INTEGER'},
         {'name':'sep_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'sep_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'sep_cleared_unused', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'sep_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'sep_officers_killed_felony', 'type':'STRING'},
         {'name':'sep_officers_killed_accident', 'type':'STRING'},
         {'name':'sep_officers_assaulted', 'type':'STRING'},
         {'name':'oct_month_in', 'type':'STRING'},
         {'name':'oct_date_last_update', 'type':'STRING'},
         {'name':'oct_card_0_type', 'type':'STRING'},
         {'name':'oct_card_1_type', 'type':'STRING'},
         {'name':'oct_card_2_type', 'type':'STRING'},
         {'name':'oct_card_3_type', 'type':'STRING'},
         {'name':'oct_card_4_type', 'type':'STRING'},
         {'name':'oct_card_0_pt', 'type':'STRING'},
         {'name':'oct_card_1_pt', 'type':'STRING'},
         {'name':'oct_card_2_pt', 'type':'STRING'},
         {'name':'oct_card_3_pt', 'type':'STRING'},
         {'name':'oct_unfounded_murder', 'type':'INTEGER'},
         {'name':'oct_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'oct_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'oct_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'oct_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'oct_unfounded_robbery', 'type':'INTEGER'},
         {'name':'oct_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'oct_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'oct_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'oct_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'oct_unfounded_assault', 'type':'INTEGER'},
         {'name':'oct_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'oct_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'oct_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'oct_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'oct_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'oct_unfounded_burglary', 'type':'INTEGER'},
         {'name':'oct_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'oct_unfounded_larceny', 'type':'INTEGER'},
         {'name':'oct_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'oct_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'oct_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'oct_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'oct_unfounded_unused', 'type':'INTEGER'},
         {'name':'oct_actual_murder', 'type':'INTEGER'},
         {'name':'oct_actual_manslaughter', 'type':'INTEGER'},
         {'name':'oct_actual_rape_total', 'type':'INTEGER'},
         {'name':'oct_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'oct_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'oct_actual_robbery', 'type':'INTEGER'},
         {'name':'oct_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'oct_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'oct_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'oct_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'oct_actual_assault', 'type':'INTEGER'},
         {'name':'oct_actual_assault_gun', 'type':'INTEGER'},
         {'name':'oct_actual_assault_knife', 'type':'INTEGER'},
         {'name':'oct_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'oct_actual_assault_hands', 'type':'INTEGER'},
         {'name':'oct_actual_assault_simple', 'type':'INTEGER'},
         {'name':'oct_actual_burglary', 'type':'INTEGER'},
         {'name':'oct_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'oct_actual_larceny', 'type':'INTEGER'},
         {'name':'oct_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_actual_auto_theft', 'type':'INTEGER'},
         {'name':'oct_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'oct_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'oct_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'oct_actual_unused', 'type':'INTEGER'},
         {'name':'oct_cleared_murder', 'type':'INTEGER'},
         {'name':'oct_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'oct_cleared_rape_total', 'type':'INTEGER'},
         {'name':'oct_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'oct_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'oct_cleared_robbery', 'type':'INTEGER'},
         {'name':'oct_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'oct_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'oct_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'oct_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'oct_cleared_assault', 'type':'INTEGER'},
         {'name':'oct_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'oct_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'oct_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'oct_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'oct_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'oct_cleared_burglary', 'type':'INTEGER'},
         {'name':'oct_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'oct_cleared_larceny', 'type':'INTEGER'},
         {'name':'oct_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'oct_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'oct_cleared_unused', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'oct_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'oct_officers_killed_felony', 'type':'STRING'},
         {'name':'oct_officers_killed_accident', 'type':'STRING'},
         {'name':'oct_officers_assaulted', 'type':'STRING'},
         {'name':'nov_month_in', 'type':'STRING'},
         {'name':'nov_date_last_update', 'type':'STRING'},
         {'name':'nov_card_0_type', 'type':'STRING'},
         {'name':'nov_card_1_type', 'type':'STRING'},
         {'name':'nov_card_2_type', 'type':'STRING'},
         {'name':'nov_card_3_type', 'type':'STRING'},
         {'name':'nov_card_4_type', 'type':'STRING'},
         {'name':'nov_card_0_pt', 'type':'STRING'},
         {'name':'nov_card_1_pt', 'type':'STRING'},
         {'name':'nov_card_2_pt', 'type':'STRING'},
         {'name':'nov_card_3_pt', 'type':'STRING'},
         {'name':'nov_unfounded_murder', 'type':'INTEGER'},
         {'name':'nov_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'nov_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'nov_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'nov_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'nov_unfounded_robbery', 'type':'INTEGER'},
         {'name':'nov_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'nov_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'nov_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'nov_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'nov_unfounded_assault', 'type':'INTEGER'},
         {'name':'nov_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'nov_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'nov_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'nov_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'nov_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'nov_unfounded_burglary', 'type':'INTEGER'},
         {'name':'nov_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'nov_unfounded_larceny', 'type':'INTEGER'},
         {'name':'nov_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'nov_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'nov_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'nov_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'nov_unfounded_unused', 'type':'INTEGER'},
         {'name':'nov_actual_murder', 'type':'INTEGER'},
         {'name':'nov_actual_manslaughter', 'type':'INTEGER'},
         {'name':'nov_actual_rape_total', 'type':'INTEGER'},
         {'name':'nov_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'nov_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'nov_actual_robbery', 'type':'INTEGER'},
         {'name':'nov_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'nov_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'nov_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'nov_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'nov_actual_assault', 'type':'INTEGER'},
         {'name':'nov_actual_assault_gun', 'type':'INTEGER'},
         {'name':'nov_actual_assault_knife', 'type':'INTEGER'},
         {'name':'nov_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'nov_actual_assault_hands', 'type':'INTEGER'},
         {'name':'nov_actual_assault_simple', 'type':'INTEGER'},
         {'name':'nov_actual_burglary', 'type':'INTEGER'},
         {'name':'nov_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'nov_actual_larceny', 'type':'INTEGER'},
         {'name':'nov_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_actual_auto_theft', 'type':'INTEGER'},
         {'name':'nov_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'nov_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'nov_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'nov_actual_unused', 'type':'INTEGER'},
         {'name':'nov_cleared_murder', 'type':'INTEGER'},
         {'name':'nov_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'nov_cleared_rape_total', 'type':'INTEGER'},
         {'name':'nov_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'nov_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'nov_cleared_robbery', 'type':'INTEGER'},
         {'name':'nov_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'nov_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'nov_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'nov_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'nov_cleared_assault', 'type':'INTEGER'},
         {'name':'nov_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'nov_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'nov_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'nov_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'nov_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'nov_cleared_burglary', 'type':'INTEGER'},
         {'name':'nov_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'nov_cleared_larceny', 'type':'INTEGER'},
         {'name':'nov_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'nov_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'nov_cleared_unused', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'nov_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'nov_officers_killed_felony', 'type':'STRING'},
         {'name':'nov_officers_killed_accident', 'type':'STRING'},
         {'name':'nov_officers_assaulted', 'type':'STRING'},
         {'name':'dec_month_in', 'type':'STRING'},
         {'name':'dec_date_last_update', 'type':'STRING'},
         {'name':'dec_card_0_type', 'type':'STRING'},
         {'name':'dec_card_1_type', 'type':'STRING'},
         {'name':'dec_card_2_type', 'type':'STRING'},
         {'name':'dec_card_3_type', 'type':'STRING'},
         {'name':'dec_card_4_type', 'type':'STRING'},
         {'name':'dec_card_0_pt', 'type':'STRING'},
         {'name':'dec_card_1_pt', 'type':'STRING'},
         {'name':'dec_card_2_pt', 'type':'STRING'},
         {'name':'dec_card_3_pt', 'type':'STRING'},
         {'name':'dec_unfounded_murder', 'type':'INTEGER'},
         {'name':'dec_unfounded_manslaughter', 'type':'INTEGER'},
         {'name':'dec_unfounded_rape_total', 'type':'INTEGER'},
         {'name':'dec_unfounded_rape_by_force', 'type':'INTEGER'},
         {'name':'dec_unfounded_rape_attempt', 'type':'INTEGER'},
         {'name':'dec_unfounded_robbery', 'type':'INTEGER'},
         {'name':'dec_unfounded_robbery_gun', 'type':'INTEGER'},
         {'name':'dec_unfounded_robbery_knife', 'type':'INTEGER'},
         {'name':'dec_unfounded_robbery_othweap', 'type':'INTEGER'},
         {'name':'dec_unfounded_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'dec_unfounded_assault', 'type':'INTEGER'},
         {'name':'dec_unfounded_assault_gun', 'type':'INTEGER'},
         {'name':'dec_unfounded_assault_knife', 'type':'INTEGER'},
         {'name':'dec_unfounded_assault_othweap', 'type':'INTEGER'},
         {'name':'dec_unfounded_assault_hands', 'type':'INTEGER'},
         {'name':'dec_unfounded_assault_simple', 'type':'INTEGER'},
         {'name':'dec_unfounded_burglary', 'type':'INTEGER'},
         {'name':'dec_unfounded_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_unfounded_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_unfounded_burglary_attempt', 'type':'INTEGER'},
         {'name':'dec_unfounded_larceny', 'type':'INTEGER'},
         {'name':'dec_unfounded_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_unfounded_auto_theft', 'type':'INTEGER'},
         {'name':'dec_unfounded_truck_bus_theft', 'type':'INTEGER'},
         {'name':'dec_unfounded_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_unfounded_total_all_fields', 'type':'INTEGER'},
         {'name':'dec_unfounded_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'dec_unfounded_unused', 'type':'INTEGER'},
         {'name':'dec_actual_murder', 'type':'INTEGER'},
         {'name':'dec_actual_manslaughter', 'type':'INTEGER'},
         {'name':'dec_actual_rape_total', 'type':'INTEGER'},
         {'name':'dec_actual_rape_by_force', 'type':'INTEGER'},
         {'name':'dec_actual_rape_attempt', 'type':'INTEGER'},
         {'name':'dec_actual_robbery', 'type':'INTEGER'},
         {'name':'dec_actual_robbery_gun', 'type':'INTEGER'},
         {'name':'dec_actual_robbery_knife', 'type':'INTEGER'},
         {'name':'dec_actual_robbery_othweap', 'type':'INTEGER'},
         {'name':'dec_actual_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'dec_actual_assault', 'type':'INTEGER'},
         {'name':'dec_actual_assault_gun', 'type':'INTEGER'},
         {'name':'dec_actual_assault_knife', 'type':'INTEGER'},
         {'name':'dec_actual_assault_othweap', 'type':'INTEGER'},
         {'name':'dec_actual_assault_hands', 'type':'INTEGER'},
         {'name':'dec_actual_assault_simple', 'type':'INTEGER'},
         {'name':'dec_actual_burglary', 'type':'INTEGER'},
         {'name':'dec_actual_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_actual_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_actual_burglary_attempt', 'type':'INTEGER'},
         {'name':'dec_actual_larceny', 'type':'INTEGER'},
         {'name':'dec_actual_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_actual_auto_theft', 'type':'INTEGER'},
         {'name':'dec_actual_truck_bus_theft', 'type':'INTEGER'},
         {'name':'dec_actual_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_actual_total_all_fields', 'type':'INTEGER'},
         {'name':'dec_actual_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'dec_actual_unused', 'type':'INTEGER'},
         {'name':'dec_cleared_murder', 'type':'INTEGER'},
         {'name':'dec_cleared_manslaughter', 'type':'INTEGER'},
         {'name':'dec_cleared_rape_total', 'type':'INTEGER'},
         {'name':'dec_cleared_rape_by_force', 'type':'INTEGER'},
         {'name':'dec_cleared_rape_attempt', 'type':'INTEGER'},
         {'name':'dec_cleared_robbery', 'type':'INTEGER'},
         {'name':'dec_cleared_robbery_gun', 'type':'INTEGER'},
         {'name':'dec_cleared_robbery_knife', 'type':'INTEGER'},
         {'name':'dec_cleared_robbery_othweap', 'type':'INTEGER'},
         {'name':'dec_cleared_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'dec_cleared_assault', 'type':'INTEGER'},
         {'name':'dec_cleared_assault_gun', 'type':'INTEGER'},
         {'name':'dec_cleared_assault_knife', 'type':'INTEGER'},
         {'name':'dec_cleared_assault_othweap', 'type':'INTEGER'},
         {'name':'dec_cleared_assault_hands', 'type':'INTEGER'},
         {'name':'dec_cleared_assault_simple', 'type':'INTEGER'},
         {'name':'dec_cleared_burglary', 'type':'INTEGER'},
         {'name':'dec_cleared_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_cleared_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_cleared_burglary_attempt', 'type':'INTEGER'},
         {'name':'dec_cleared_larceny', 'type':'INTEGER'},
         {'name':'dec_cleared_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_auto_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_truck_bus_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_total_all_fields', 'type':'INTEGER'},
         {'name':'dec_cleared_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'dec_cleared_unused', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_murder', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_manslaughter', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_rape_total', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_rape_by_force', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_rape_attempt', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_robbery', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_robbery_gun', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_robbery_knife', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_robbery_othweap', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_robbery_strong_arm', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_assault', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_assault_gun', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_assault_knife', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_assault_othweap', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_assault_hands', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_assault_simple', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_burglary', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_burglary_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_burglary_no_forcible_entry', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_burglary_attempt', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_larceny', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_motor_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_auto_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_truck_bus_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_other_vehicle_theft', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_total_all_fields', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_larceny_under_50_dollars', 'type':'INTEGER'},
         {'name':'dec_cleared_u18_unused', 'type':'INTEGER'},
         {'name':'dec_officers_killed_felony', 'type':'STRING'},
         {'name':'dec_officers_killed_accident', 'type':'STRING'},
         {'name':'dec_officers_assaulted', 'type':'STRING'}
        ]
      } 

        upload = MediaFileUpload('reta_%s.csv'%(i), # sample.csv is from the bigquerye2e ,
                           mimetype='application/octet-stream',
                           # This enables resumable uploads.
                           resumable=False) #post mode s
        file='/notebooks/'
        os.remove(file+'reta_%s.csv'%(i))
  # End of job configuration.
        start_and_wait(service.jobs(),PROJECT_ID,load_config,media_body=upload)

if __name__ == '__main__':
    main()   

Go to the following link in your browser: https://accounts.google.com/o/oauth2/auth?redirect_uri=https%3A%2F%2Fwww.example.com%2Foauth2callback&client_id=391362830666-5o9iss6ptg667crq884sc199l9vosbmk.apps.googleusercontent.com&response_type=code&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&access_type=offline
Enter verification code: 4/VBEEo2nPgFMSPA0ufDU0MRN1lqAlGI_5MAurC5pgrvU#
Enter the Project_IDfair-catbird-99215
Enter the Dataset_IDabc
Authorisation successfully completed
{
  "jobReference": {
    "projectId": "fair-catbird-99215",
    "jobId": "ch06_1439432307"
  },
  "statistics": {
    "creationTime": "1439432448940"
  },
  "configuration": {
    "load": {
      "destinationTable": {
        "tableId": "reta_1969",
        "projectId": "fair-catbird-99215",
        "datasetId": "abc"
      },
      "schema": {
        "fields": [
          {
            "name": "id",
            "type": "STRING"
          },
          {
            "name": "state",
            "typ

AccessTokenRefreshError: invalid_grant

In [30]:
import os

os.getcwd()

'/notebooks'